In [ ]:
from IPython.core.display import HTML
with open ("../style.css", "r") as file:
    css = file.read()
HTML(css)

# Reverse Mode Automatic Differentiation

We demonstrate reverse mode AD with the function
$$ f(x_1, x_2) = \sin(x_1 + x_2) \cdot \cos(x_1 - x_2) + (x_1 + x_2) \cdot (x_1 - x_2) $$
To compute the function step by step, we introduce the following auxiliary variables:
* $v_1 := x_1 + x_2$,
* $v_2 := x_1 - x_2$,
* $v_3 := \sin(v_1)$,
* $v_4 := \cos(v_2)$,
* $v_5 := v_3 \cdot v_4$,
* $v_6 := v_1 \cdot v_2$,
* $y   := v_5 + v_6$.

In [ ]:
def f(x1, x2):
    return sin(x1 + x2) * cos(x1 - x2) + (x1 + x2) * (x1 - x2)

The central notion of *reverse-mode* [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation) is the notion of a *computational graph*, which is a list of *computational nodes*.
There are four types of computational nodes:
- A *variable node* is tuple of length 1 of the form   
  $$\langle x, \rangle$$
  where $x$ is a variable from a given set of variables $\{x_1,\cdots,x_k\}$.
- A *constant node* is a pair of the form
  $$\langle n, r\rangle$$
  where $n$ is the *name* of the node and $r$ is a floating point number.  
  
  This node is interpreted as the assignment
  $$n \;\mathtt{:=}\; r$$
  The name $n$ is a string that can be understood as the name of an auxiliary variable.
- A *unary node* is a tuple of the form
  $$\langle n, f, a\rangle$$
  where $n$ is the *name* of the node, $f$ is an unary function symbol from the set
  $$\{ \mathrm{sqrt}, \exp, \ln, \sin, \cos, \arctan \}$$
  and $a$ is the name of another node occurring in the list.  
  
  This node is interpreted as the assignment
  $n \;\mathtt{:=}\; f(a)$.        
- A *binary node* is a tuple of the form
  $$\langle n, o, a_1, a_2\rangle$$
  where $n$ is the *name* of the node, $o$ is a binary operator from the set
  $$\{ +, -, *, / \}$$
  and $a_1$ and $a_2$ are the names of other computational nodes.
        
  This node is interpreted as the assignment
  $$n \;\mathtt{:=}\; a_1\; o\,\; a_2 $$
  The name $n$ is a string that can be understood as the name of an auxiliary variable.

---

The computational graph `GC` defined below implements the function `f` that is defined as follows:
$$ f(x_1, x_2) = \sin(x_1 + x_2) \cdot \cos(x_1 - x_2) + (x_1 + x_2) \cdot (x_1 - x_2) $$

In [ ]:
CG = [ ('x1', ),
       ('x2', ),
       ('v1', '+',   'x1', 'x2'),
       ('v2', '-',   'x1', 'x2'),
       ('v3', 'sin', 'v1'),
       ('v4', 'cos', 'v2'),
       ('v5', '*',   'v3', 'v4'),
       ('v6', '*',   'v1', 'v2'),
       ('y',  '+',   'v5', 'v6')
     ]

In [ ]:
import graphviz as gv

The function `render(CG)` takes a computational graph `CG` as input and renders it graphically via `graphviz`.

In [ ]:
def render(CG):
    cg = gv.Digraph()
    cg.attr(rankdir='LR')
    for node in CG:
        shape = 'rectangle'
        match node:
            case (v, ):
                label = f'{v}'
                shape = 'circle'
            case (v, r):
                label = f'{v} := {r}'
            case (v, op, a1, a2):
                label = f'{v} := {a1} {op} {a2}'
            case (v, f, a):
                label = f'{v} :=  {f}({a})'
        cg.node(v, label=label, shape=shape)
    for node in CG:
        match node:
            case (v, _, a1, a2):
                cg.edge(a1, v)
                cg.edge(a2, v)
            case (v, _, a):
                cg.edge(a, v)
    return cg

$$ f(x_1, x_2) = \sin(x_1 + x_2) \cdot \cos(x_1 - x_2) + (x_1 + x_2) \cdot (x_1 - x_2) $$

In [ ]:
dot = render(CG)
dot

In [ ]:
dot.save(filename='cg.dot')

In [ ]:
!cat cg.dot || type cg.dot

In [ ]:
import math

The function `eval_graph` takes two arguments:
* `CG` is a computational graph,
* `Values` is a dictionary assigning values to variable names.

In [ ]:
def eval_graph(CG, Values):
    for node in CG:
        match node:
            case (v, ):
                pass     # input variables are already defined
            case (v, r):
                Values[v] = r
            case (v, '+', a1, a2):
                Values[v] = Values[a1] + Values[a2]
            case (v, '-', a1, a2):
                Values[v] = Values[a1] - Values[a2]
            case (v, '*', a1, a2):
                Values[v] = Values[a1] * Values[a2]
            case (v, '/', a1, a2):
                Values[v] = Values[a1] / Values[a2]
            case (v, 'sqrt', a):
                Values[v] = math.sqrt(Values[a])            
            case (v, 'exp', a):
                Values[v] = math.exp(Values[a])
            case (v, 'log', a):
                Values[v] = math.log(Values[a])
            case (v, 'sin', a):
                Values[v] = math.sin(Values[a])
            case (v, 'cos', a):
                Values[v] = math.cos(Values[a])
            case (v, 'atan', a):
                Values[v] = math.atan(Values[a])
    return Values['y']

In [ ]:
eval_graph(CG, { 'x1': math.pi/4, 'x2': math.pi/4 })

The function `add_to_dictionary` takes three arguments:
* `D` is a dictionary.  

   This dictionary maps keys to sets of values.
* `key` is a key for this dictionary.
* `value` is a value that is added to the set `D[key]`.

The function does not return a value but instead changes the dictionary `D` such that the following equation holds:
$$ \texttt{value} \in \texttt{D}[\texttt{key}] $$

In [ ]:
def add_to_dictionary(D, key, value):
    if key in D:
        D[key] |= { value } # operator |= performs a union
    else:
        D[key]  = { value }

Given a *computational graph* `CG`, the function `parents` returns a dictionary `Parents` such that
for every node name `n` occurring in `CG` we have that `Parents[n]` is the set of nodes that are parents 
of the node labeled with `n`. 
It returns the dictionary `Parents`.  
* If `CG` contains a node of the form $(p, f, a)$, then $p$ is a parent of $a$.
* If `CG` contains a node of the form $(p, o, a_1, a_2)$, then $p$ is a parent of both 
  $a_1$ and $a_2$.
  
It is important to understand that a node can have an arbitrary number of parents.

In [ ]:
def parents(CG):
    Parents = {}
    for node in CG:
        match node:
            case (p, _, a):
                add_to_dictionary(Parents, a, p)
            case (p, _, a1, a2):
                add_to_dictionary(Parents, a1, p)
                add_to_dictionary(Parents, a2, p)
    return Parents

In [ ]:
render(CG)

In [ ]:
parents(CG)

Given a computational graph `CG` the function `node_dictionary` returns a dictionary that maps the names of the nodes to the nodes themselves. 

In [ ]:
def node_dictionary(CG):
    D = {}
    for node in CG:
        name    = node[0]
        D[name] = node
    return D

In [ ]:
node_dictionary(CG)

The function `partial_derivative` takes three arguments:
* `Node` is a computational node,
* `arg` is the name of a node occurring as argument in `Node`, 
* `Values` is a dictionary that stores a value for every node name.

The function computes the partial derivative of `Node` w.r.t. `arg`.

In [ ]:
def partial_derivative(Node, arg, Values):
    match Node:
        case n, '+', a1, a2:
            if arg == a1 == a2:
                return 2
            if arg == a1 or arg == a2:
                return 1
            else:
                assert False, f'partial_derivative({Node}, {arg})'
        case n, '-', a1, a2:
            if arg == a1 == a2:
                return 0
            if arg == a1:
                return 1
            if arg == a2:
                return -1
            else:
                assert False, f'partial_derivative({Node}, {arg})'
        case n, '*', a1, a2:
            if arg == a1 == a2:
                return 2 * Values[a1]
            if arg == a1:
                return Values[a2]
            if arg == a2:
                return Values[a1]
            else:
                assert False, f'partial_derivative({Node}, {arg})'
        case n, '/', a1, a2:
            if arg == a1 == a2:
                return 0
            if arg == a1:
                return 1 / Values[a2]
            if arg == a2:
                return -Values[a1] / Values[a2] ** 2
            else:
                assert False, f'partial_derivative({Node}, {arg})'
        case n, 'sqrt', a:
            return 0.5 / math.sqrt(Values[a])
        case n, 'exp', a:
            return math.exp(Values[a])
        case n, 'log', a:
            return 1 / Values[a]
        case n, 'sin', a:
            return math.cos(Values[a])
        case n, 'cos', a:
            return -math.sin(Values[a])
        case n, 'atan', a:
            return 1 / (1 + Values[a]**2)

Assume that `CG` is a *computational graph* such that the *output node* is labelled with then name `y`. If $v$ is any node in `CG`, then the *adjoint* of $v$, is defined
as the partial derivative of the output variable `y` w.r.t. $v$, i.e.
$$\bar{v} := \frac{\displaystyle\partial \mathtt{y}}{\displaystyle\partial v}.$$
The adjoint of $v$ is written as $\bar{v}$. 

If $p_1, \cdots, p_k$ are all the parents of the node $v$ in `CG`, then 
according to the chain rule of multivariable calculus the adjoint $\bar{v}$
can be computed as follows:
$$\bar{v} = \frac{\displaystyle\partial y}{\displaystyle\partial v} 
              = \sum\limits_{i=1}^k \frac{\displaystyle\partial y}{\displaystyle\partial p_i} \cdot \frac{\displaystyle\partial p_i}{\displaystyle\partial v}
              = \sum\limits_{i=1}^k \bar{p}_i \cdot \frac{\displaystyle\partial p_i}{\displaystyle\partial v}
  $$
---  
The function `adjoints` takes two arguments:
- `CG` is a *computational graph* `CG`.
- `Values` is a dictionary.  This dictionary maps the names of the *input nodes*
  occurring in `CG` to their corresponding values.
  
The function returns a dictionary `Adjoints` such that for every name $v$ of a 
node occurring in `CG` we have that `Adjoints[v]` is the *adjoin* of $v$.

In [ ]:
def adjoints(CG, Values):
    eval_graph(CG, Values)
    NodeDict = node_dictionary(CG)
    Parents  = parents(CG)
    n        = len(CG)
    Adjoints = {}
    Adjoints['y'] = 1
    for Node in reversed(CG[:-1]):
        name   = Node[0]
        result = 0
        for parent_name in Parents[name]:
            parent_node = NodeDict[parent_name]
            result += Adjoints[parent_name] * partial_derivative(parent_node, name, Values)
        Adjoints[name] = result
    return Adjoints

In [ ]:
adjoints(CG, { 'x1': math.pi/4, 'x2': math.pi/4 })

The function `autodiff` takes three arguments:
- `CG` is a computational graph.
- `Values` is a dictionary mapping the input variables of `CG` to values.
- `x` is the name of an input variable.

If the computaional graph `CG` computes the function $f(x_1,\cdots,x_n)$, then
`autodiff` returns the paritial derivative $\frac{\displaystyle\partial f}{\displaystyle\partial x}$. 

In [ ]:
def autodiff(CG, Values, x):
    Adjoints = adjoints(CG, Values)
    return Adjoints[x]

In [ ]:
autodiff(CG, { 'x1': math.pi/4, 'x2': math.pi/4 }, 'x1')

If 
$$f(x_1,x_2) = \sin(x_1 + x_2) \cdot \cos(x_1 - x_2) + (x_1 + x_2) \cdot (x_1 - x_2), $$
we have
$$\displaystyle \frac{\partial\; f}{\partial x_1} 
= -\sin(x_1-x_2) \cdot \sin(x_1 + x_2)+ \cos(x_1-x_2) \cdot \cos(x_1 + x_2) + 2 \cdot x_1. $$
Furthermore,
$$\displaystyle  -\sin(\frac{\pi}{4}-\frac{\pi}{4}) \cdot \sin(\frac{\pi}{4}+\frac{\pi}{4})+ \cos(\frac{\pi}{4}-\frac{\pi}{4}) \cdot \cos(\frac{\pi}{4}+\frac{\pi}{4}) + 2 \cdot \frac{\pi}{4} = \frac{\pi}{2}.$$
Hence the computed value in the cell above is correct.